Team members are junseonglee11 (@junseonglee11), Ayaan Jang(@ayaanjang).
In this notebook, We just combined many great public notebooks and fine-tuned the model.  
And We usually executed this notebook on the Google Colab TPU with Google Drive (for fast saving). 

# References

1. Robin smith's: notebooks:   
https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-inference  
https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-training-tpu  
https://www.kaggle.com/code/rsmits/tensorflow-lstm-model-data-preprocessor/notebook  
Robin Smith's reference  
https://www.kaggle.com/code/seungmoklee/lstm-preprocessing-point-picker
   I modified his notebook   
   a. Converted the dataset to TFRecords  
   b. Additional inputs and some preprocessing  
   c. Changed the model (more RNN layers, GRU --> LSTM, GELU activation, rectified adam optimizer)

2. Robert Hatch's notebook  
https://www.kaggle.com/code/roberthatch/lb-1-183-lightning-fast-baseline-with-polars  
It was crucial to improve our score.
Used the results of this notebook as additional inputs in our model.

TFRecord dataset generation: https://www.kaggle.com/code/junseonglee11/icecube-data-to-tfrecord-v2-1

# Google Colab Part

1. Connect to Google drive and install require libraries  
2. Save weights to Google drive with filename = save_name

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

#from google.colab import files
#files.upload()

In [6]:
#!pip -qq install kaggle
#!pip -qq install datasets
!pip -qq install tensorflow-addons
!pip -qq install wandb

In [7]:
save_name = '230420_LSTM_90file_line-aux_2layer_radam_gelu_drop_f0_5th'
IS_WANDB = False
WANDB_KEY = None

# Import libraries

In [21]:
# Import
import glob
import numpy as np
import os
import gc
import tensorflow as tf
import random
from tqdm.notebook import tqdm
from kaggle_datasets import KaggleDatasets
from keras import backend as K
import itertools
import math
from matplotlib import pyplot as plt
import tensorflow_addons as tfa
import wandb
from wandb.keras import WandbCallback

#for wandb logging
if (WANDB_KEY is not None):
    wandb.login(key = WANDB_KEY)
    os.environ["WANDB_SILENT"] = "true"
    IS_WANDB = True
    print('Use WandB for logging!')

tf.config.optimizer.set_jit(True)
#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.experimental.output_all_intermediates(True)
#tf.compat.v1.enable_eager_execution()
#tf.config.run_functions_eagerly(True)
#tf.compat.v1.enable_eager_execution()
#tf.data.experimental.enable_debug_mode()

# TPU setting

In [9]:
# Configure Strategy. Assume TPU...if not set default for GPU
tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.MirroredStrategy()
    

N_REPLICAS = strategy.num_replicas_in_sync
print(f'N_REPLICAS: {N_REPLICAS}, IS_TPU: {tpu is not None}')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


N_REPLICAS: 8, IS_TPU: True


# Constants

In [10]:
VERBOSE = 1

# Training
N_FEATS = 9
N_WARMUP_EPOCHS = 10
LR_MAX = 0.0005# *N_REPLICAS
LR_MIN = LR_MAX/4
STEP_SIZE = 4

WD_RATIO = 0.01
N_EPOCHS = 5
n_folds = 10
selected_fold = 0
batch_size = 896*N_REPLICAS    # Local Training: 2048

# Model Parameters
pulse_count = 96
feature_count = N_FEATS
lstm_units = 192
bin_num = 32

config = {
    'project': 'IceCube',    
    'name'  : 'cnn0_LSTM_90file_line-2layer_radam_gelu_drop_f0_5th',
    'group'  : 'line_fit',
    'N_FEATS': N_FEATS,
    'N_WARMUP_EPOCHS': N_WARMUP_EPOCHS,
    'LR_MAX' : LR_MAX,
    'LR_MIN' : LR_MIN,
    'STEP_SIZE': STEP_SIZE,
    'WD_RATIO': WD_RATIO,
    'N_EPOCHS': N_EPOCHS,
    'N_FOLDS' : n_folds,
    'SELECTED_FOLD': selected_fold,
    'BATCH_SIZE': batch_size,
    'pulse_count': pulse_count,
    'feature_count':N_FEATS,
    'lstm_units': lstm_units,
    'bin_num' : bin_num,
    'save_name': save_name,
    'N_FILES'  : 660,
    'IS_TPU': tpu is not None
    }



In [11]:
# Set Seed
seed = 4242
tf.random.set_seed(seed)
random.seed(seed)
np.random.seed(seed)

## Define Azimuth and Zenith Bins

# TFRecord dataset
230317 junseonglee11 for full data training  
Converted azimuth and zenith values (float) to category using keras backend operations

In [14]:
tensor_azimuth_edges = tf.constant(azimuth_edges, dtype = tf.float32)
tensor_zenith_edges = tf.constant(zenith_edges,dtype = tf.float32)

def decode_tfrecord(record_bytes):
    features = tf.io.parse_single_example(record_bytes, {
        'event_pulses': tf.io.FixedLenFeature([], tf.string),
        'fitted_azimuth': tf.io.FixedLenFeature([], tf.float32),               
        'fitted_zenith' : tf.io.FixedLenFeature([], tf.float32),
        'origin_azimuth': tf.io.FixedLenFeature([], tf.float32),               
        'origin_zenith' : tf.io.FixedLenFeature([], tf.float32),
    })        

    event_pulses = tf.io.parse_tensor(features['event_pulses'], out_type=tf.float16)
    event_pulses = tf.cast(event_pulses, tf.float16)
    event_pulses = tf.reshape(event_pulses, [96, 9])
    event_pulses = tf.slice(event_pulses, [0, 0], [96, feature_count])
    
    fitted_azimuth = features['fitted_azimuth']
    fitted_azimuth = tf.cast(fitted_azimuth, tf.float16)
    fitted_zenith  = features['fitted_zenith']
    fitted_zenith  = tf.cast(fitted_zenith, tf.float16)

    fitted_targets = tf.stack([fitted_azimuth, fitted_zenith])
    
    origin_azimuth = features['origin_azimuth']    
    origin_azimuth = tf.cast(origin_azimuth, tf.float32)
    
    origin_zenith =  features['origin_zenith']
    origin_zenith = tf.cast(origin_zenith, tf.float32)
    
    
    azimuth_array = tf.repeat(origin_azimuth, bin_num+1)
    azimuth_array = K.greater(origin_azimuth, tensor_azimuth_edges)
    azimuth_array = tf.cast(azimuth_array, tf.int32)
    azimuth = K.sum(azimuth_array, axis = 0)-1
    
    #for encodeing reproduce            
    zenith_array = tf.repeat(origin_zenith, (bin_num+1))
    zenith_array = K.greater(origin_zenith, tensor_zenith_edges)
    zenith_array = tf.cast(zenith_array, tf.int32)
    zenith = K.sum(zenith_array, axis = 0)-1
    

    
    encoded_angle = azimuth*(bin_num)+zenith
    origin_targets = tf.stack([origin_azimuth, origin_zenith])
    
    
    return {'event_pulses': event_pulses, 'fitted_targets': fitted_targets }, {
            'encoded_angle': encoded_angle}

In [15]:
# Tensorflow AUTO flag
AUTO = tf.data.experimental.AUTOTUNE

def get_dataset(tfrecords, bs=batch_size, val=False, debug=False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    
    # Initialize dataset with TFRecords
    dataset = tf.data.TFRecordDataset(tfrecords, num_parallel_reads=AUTO, compression_type='GZIP')
    
    # Decode mapping
    dataset = dataset.map(decode_tfrecord, num_parallel_calls=AUTO)

    if not val:                
        dataset = dataset.with_options(ignore_order)
        if not debug:
            dataset = dataset.shuffle(2000000)
        dataset = dataset.repeat()        

    dataset = dataset.batch(bs, drop_remainder= not val)
    dataset = dataset.prefetch(AUTO)    
    return dataset

In [35]:
TFRECORDS_FILE_PATHS = sorted(glob.glob('/kaggle/input/icecube-tfrecords-dataset-v3-*/*'))

file_inds = np.zeros(len(TFRECORDS_FILE_PATHS), int)
counter = 0
for file_path in TFRECORDS_FILE_PATHS:
    tmp = file_path.split('/')[-1][:-9]
    file_inds[counter] = int(tmp)    
    counter+=1
file_ind_order = np.argsort(file_inds)

print('# of TFRecords: %d'%len(TFRECORDS_FILE_PATHS))
TFRECORDS_TRAIN =[]
TFRECORDS_VALID =[]
for i in range(0, config['N_FILES']):#len(TFRECORDS_FILE_PATHS)):
    if(i%n_folds == selected_fold):
        TFRECORDS_VALID.append(TFRECORDS_FILE_PATHS[file_ind_order[i]])
        print(TFRECORDS_FILE_PATHS[file_ind_order[i]])
    else:
        TFRECORDS_TRAIN.append(TFRECORDS_FILE_PATHS[file_ind_order[i]])
                              
print('# of train TFRECORDS: %d'%len(TFRECORDS_TRAIN))
print('# of valid TFRECORDS: %d'%len(TFRECORDS_VALID))
TRAIN_STEPS_PER_EPOCH = len(TFRECORDS_TRAIN)*200000//batch_size
print('Train steps per epoch: %d'%TRAIN_STEPS_PER_EPOCH)

# of TFRecords: 660
/kaggle/input/icecube-tfrecords-dataset-v3-1/1.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/11.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/21.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/31.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/41.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/51.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/61.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/71.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/81.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/91.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/101.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-1/111.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-2/121.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-2/131.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-2/141.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-2/151.tfrecord
/kaggle/input/icecube-tfrecords-dataset-v3-2/161.tfrecord
/kagg

# Model and callbacks definition

In [36]:
def create_model():
    with strategy.scope(): 
        inputs_1 = tf.keras.layers.Input((pulse_count, feature_count))
        inputs_2 = tf.keras.layers.Input((2,))

        x = tf.keras.layers.Masking(mask_value = 0., input_shape = (pulse_count, feature_count))(inputs_1)

        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences = True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(256, activation = 'gelu')(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.concat([x, inputs_2], axis = 1)
        x = tf.keras.layers.Dense(258, activation = 'gelu')(x)
        x = tf.keras.layers.Dense(258, activation = 'gelu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(258, activation = 'gelu')(x)

        outputs = tf.keras.layers.Dense(bin_num**2, activation = 'softmax')(x)

        # Finalize Model
        model = tf.keras.models.Model(inputs = {'event_pulses': inputs_1, 'fitted_targets': inputs_2}, outputs = {'encoded_angle': outputs})

        # Compile model
        model.compile(loss = 'sparse_categorical_crossentropy',
                      #optimizer= tf.keras.optimizers.Adam(learning_rate = LR_MAX),
                      optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LR_MAX),
                      metrics = ['accuracy'])

        # Show Model Summary
        model.summary()

        return model

In [37]:
filepath = "epoch{epoch:02d}-val_acc{val_accuracy:.5f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=False)
callbacks = [checkpoint]

In [38]:
if(IS_WANDB):
    wandb.init(project = config['project'], name = f"{config['name']}",
                   group = config['group'], config = config)
    wandb_callback = WandbCallback(
        monitor="val_accuracy", save_model=(False), mode = 'max'
    )
    callbacks.append(wandb_callback)

In [ ]:
train_dataset = get_dataset(TFRECORDS_TRAIN, val=False, debug=False)
valid_dataset = get_dataset(TFRECORDS_VALID, val=True, debug=False)

K.clear_session()

model = create_model()
                    
history = model.fit(
        train_dataset,
        steps_per_epoch = TRAIN_STEPS_PER_EPOCH ,        
        validation_data = valid_dataset,
        epochs = N_EPOCHS,
        verbose = VERBOSE,
        callbacks = callbacks
    )
model.save('last_model.h5')

#del model    
#K.clear_session()
#gc.collect()
    
wandb.finish()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 9)]      0           []                               
                                                                                                  
 masking (Masking)              (None, 96, 9)        0           ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 96, 384)      310272      ['masking[0][0]']                
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 96, 384)     886272      ['bidirectional[0][0]']          
 )                                                                                            

2023-04-20 00:56:08.811038: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-04-20 00:56:10.261886: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


   87/16573 [..............................] - ETA: 1:41:56 - loss: 6.9342 - accuracy: 0.0011

In [ ]:
del model
gc.collect()
K.clear_session()

# Save to Google Drive (for Colab)

In [ ]:
import glob
import os
import shutil 

try:    
    file_list = glob.glob('*h5')
    print(file_list)


    os.mkdir(save_name)
    for i in range(0, len(file_list)):
        shutil.copyfile(file_list[i], '%s/%s'%(save_name, file_list[i]))
    shutil.make_archive('/content/drive/MyDrive/%s'%save_name, 'gztar', save_name)

except: 
    print('It is not Colab notebook')